In [ ]:
import os, shutil, csv

original_dataset_dir = '/Users/mithyyin/Documents/GitHub/TeamEve/Classfication_small_datasets_inception_v3/waste_original_dataset' #directory name of your biendata
#original_dataset_dir =r'C:\Users\oscarscaro\Documents\GitHub\TeamEve\Classfication_small_datasets_inception_v3\images_withoutrect'
base_dir = './data_small' #create a directory for the data subset

#os.mkdir(base_dir)

#creating a new folder for each set
train_dir = os.path.join(base_dir, 'train') 
#os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation') 
#os.mkdir(validation_dir) 
test_dir = os.path.join(base_dir, 'test') 
#os.mkdir(test_dir)

In [ ]:
batch_size = 16 #try 32, 128
epoch = 100 # try 50, 100. 
#data_augmentation = True

In [ ]:
#Pre-process steps 2, 
train_datagen = ImageDataGenerator(rescale=1./255)

#changed here, major changes
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, #train_dir is the path where you store all the validaiton folder, chnage this
    target_size = (img_rows,img_cols), #try 1920,1080
    batch_size = batch_size,
    class_mode = 'categorical')

#tr_crops = crop_generator(train_generator)

validation_generator = test_datagen.flow_from_directory( #debug here
    validation_dir, #train_dir is the path where you store all the validation folder, change this
    target_size = (img_cols,img_cols),
    batch_size = batch_size,
    class_mode = 'categorical')

#val_crops = crop_generator(val_gen)

In [1]:
#import keras libraries
from keras.layers import Activation
from keras.layers import Add
from keras.layers import AveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import Concatenate
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling2D
from keras.layers import Input
from keras.layers import Lambda
from keras.layers import MaxPooling2D
from keras.layers import ZeroPadding2D
from keras.models import Model
from keras.optimizers import Adam
from keras.initializers import glorot_uniform

Using TensorFlow backend.


In [2]:
# split funciton
def split(inputs, cardinality):
    inputs_channels = inputs.shape[3]
    group_size = inputs_channels // cardinality    
    groups = list()
    for number in range(1, cardinality+1):
        begin = int((number-1)*group_size)
        end = int(number*group_size)
        block = Lambda(lambda x:x[:,:,:,begin:end])(inputs)
        groups.append(block)
    return groups

In [3]:
#transform and merge operations
def transform(groups, filters, strides, stage, block):
    f1, f2 = filters    
    conv_name = "conv2d-{stage}{block}-branch".format(stage=str(stage), block=str(block))
    bn_name = "batchnorm-{stage}{block}-branch".format(stage=str(stage), block=str(block))
    
    transformed_tensor = list()
    i = 1
    
    for inputs in groups:
        # first conv of the transformation phase
        x = Conv2D(filters=f1, kernel_size=(1,1), strides=strides, padding="valid", 
                   name=conv_name+'1a_split'+str(i), kernel_initializer=glorot_uniform(seed=0))(inputs)
        x = BatchNormalization(axis=3, name=bn_name+'1a_split'+str(i))(x)
        x = Activation('relu')(x)

        # second conv of the transformation phase
        x = Conv2D(filters=f2, kernel_size=(3,3), strides=(1,1), padding="same", 
                   name=conv_name+'1b_split'+str(i), kernel_initializer=glorot_uniform(seed=0))(x)
        x = BatchNormalization(axis=3, name=bn_name+'1b_split'+str(i))(x)
        x = Activation('relu')(x)
        
        # Add x to transformed tensor list
        transformed_tensor.append(x)
        i+=1
        
    # Concatenate all tensor from each group
    x = Concatenate(name='concat'+str(stage)+''+block)(transformed_tensor)
    
    return x

In [4]:
def transition(inputs, filters, stage, block):
    x = Conv2D(filters=filters, kernel_size=(1,1), strides=(1,1), padding="valid", 
                   name='conv2d-trans'+str(stage)+''+block, kernel_initializer=glorot_uniform(seed=0))(inputs)
    x = BatchNormalization(axis=3, name='batchnorm-trans'+str(stage)+''+block)(x)
    x = Activation('relu')(x)
    
    return x

In [5]:
#implementing residual blocks
def identity_block(inputs, filters, cardinality, stage, block, strides=(1,1)):
    
    conv_name = "conv2d-{stage}{block}-branch".format(stage=str(stage),block=str(block))
    bn_name = "batchnorm-{stage}{block}-branch".format(stage=str(stage),block=str(block))
    
    #save the input tensor value
    x_shortcut = inputs
    x = inputs
    
    f1, f2, f3 = filters
    
    # divide input channels into groups. The number of groups is define by cardinality param
    groups = split(inputs=x, cardinality=cardinality)
    
    # transform each group by doing a set of convolutions and concat the results
    f1 = int(f1 / cardinality)
    f2 = int(f2 / cardinality)
    x = transform(groups=groups, filters=(f1, f2), strides=strides, stage=stage, block=block)
    
    # make a transition by doing 1x1 conv
    x = transition(inputs=x, filters=f3, stage=stage, block=block)
    
    # Last step of the identity block, shortcut concatenation
    x = Add()([x,x_shortcut])
    x = Activation('relu')(x)
    
    return x

In [6]:
#The input and output are then of different dimensions. In this case, we use the downsampling() function in which we use the Projection Shortcut to match dimensions before doing the Add() operation.
def downsampling(inputs, filters, cardinality, strides, stage, block):
    
    # useful variables
    conv_name = "conv2d-{stage}{block}-branch".format(stage=str(stage), block=str(block))
    bn_name = "batchnorm-{stage}{block}-branch".format(stage=str(stage), block=str(block))
    
    # Retrieve filters for each layer
    f1, f2, f3 = filters
    
    # save the input tensor value
    x_shortcut = inputs
    x = inputs
    
    # divide input channels into groups. The number of groups is define by cardinality param
    groups = split(inputs=x, cardinality=cardinality)
    
    # transform each group by doing a set of convolutions and concat the results
    f1 = int(f1 / cardinality)
    f2 = int(f2 / cardinality)
    x = transform(groups=groups, filters=(f1, f2), strides=strides, stage=stage, block=block)
    
    # make a transition by doing 1x1 conv
    x = transition(inputs=x, filters=f3, stage=stage, block=block)
    
    # Projection Shortcut to match dimensions 
    x_shortcut = Conv2D(filters=f3, kernel_size=(1,1), strides=strides, padding="valid", 
               name='{base}2'.format(base=conv_name), kernel_initializer=glorot_uniform(seed=0))(x_shortcut)
    x_shortcut = BatchNormalization(axis=3, name='{base}2'.format(base=bn_name))(x_shortcut)
    
    # Add x and x_shortcut
    x = Add()([x,x_shortcut])
    x = Activation('relu')(x)
    
    return x

In [7]:
#Resnext-50
def ResNeXt50(input_shape, classes):
    
    # Transform input to a tensor of shape input_shape 
    x_input = Input(input_shape)
    
    # Add zero padding
    x = ZeroPadding2D((3,3))(x_input)
    
    # Initial Stage. Let's say stage 1
    x = Conv2D(filters=64, kernel_size=(7,7), strides=(2,2), 
               name='conv2d_1', kernel_initializer=glorot_uniform(seed=0))(x)
    x = BatchNormalization(axis=3, name='batchnorm_1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3,3), strides=(2,2))(x)
    
    # Stage 2
    x = downsampling(inputs=x, filters=(128,128,256), cardinality=4, strides=(2,2), stage=2, block="a")
    x = identity_block(inputs=x, filters=(128,128,256), cardinality=4, stage=2, block="b")
    x = identity_block(inputs=x, filters=(128,128,256), cardinality=4, stage=2, block="c")
    
    
    # Stage 3
    x = downsampling(inputs=x, filters=(256,256,512), cardinality=4, strides=(2,2), stage=3, block="a")
    x = identity_block(inputs=x, filters=(256,256,512), cardinality=4, stage=3, block="b")
    x = identity_block(inputs=x, filters=(256,256,512), cardinality=4, stage=3, block="c")
    x = identity_block(inputs=x, filters=(256,256,512), cardinality=4, stage=3, block="d")
    
    
    # Stage 4
    x = downsampling(inputs=x, filters=(512,512,1024), cardinality=4, strides=(2,2), stage=4, block="a")
    x = identity_block(inputs=x, filters=(512,512,1024), cardinality=4, stage=4, block="b")
    x = identity_block(inputs=x, filters=(512,512,1024), cardinality=4, stage=4, block="c")
    x = identity_block(inputs=x, filters=(512,512,1024), cardinality=4, stage=4, block="d")
    x = identity_block(inputs=x, filters=(512,512,1024), cardinality=4, stage=4, block="e")
    x = identity_block(inputs=x, filters=(512,512,1024), cardinality=4, stage=4, block="f")
    
    
    # Stage 5
    x = downsampling(inputs=x, filters=(1024,1024,2048), cardinality=4, strides=(2,2), stage=5, block="a")
    x = identity_block(inputs=x, filters=(1024,1024,2048), cardinality=4, stage=5, block="b")
    x = identity_block(inputs=x, filters=(1024,1024,2048), cardinality=4, stage=5, block="c")
    
    
    # Average pooling
    x = AveragePooling2D(pool_size=(2,2), padding="same")(x)
    
    # Output layer
    x = Flatten()(x)
    x = Dense(classes, activation="softmax", kernel_initializer=glorot_uniform(seed=0), 
              name="fc{cls}".format(cls=str(classes)))(x)
    
    # Create the model
    model = Model(inputs=x_input, outputs=x, name="resnet50")
    
    return model

In [8]:
model = ResNeXt50(input_shape=(299,299,3), classes=204)

In [9]:
optimizer = Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 305, 305, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 150, 150, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batchnorm_1 (BatchNormalization (None, 150, 150, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation

In [ ]:
# model checkpoint

#change the name of the model
filepath="waste_sort_resnext_50.h5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', 
                        verbose=1, save_best_only=True, mode='max')

# early stopping added in
# should not be a problem if you had the latest version of keras
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=150, restore_best_weights=True)
callbacks_list = [checkpoint, early]

In [ ]:
#train the model
history = model.fit_generator(
train_generator,
steps_per_epoch=100,
epochs=epoch,
verbose=1,
validation_data=validation_generator,
validation_steps=50,
callbacks = callbacks_list) #change here

In [ ]:
#visualize the data

# Training plots
epochs = [i for i in range(1, len(history.history['loss'])+1)]

plt.plot(epochs, history.history['loss'], color='blue', label="training_loss")
plt.plot(epochs, history.history['val_loss'], color='red', label="validation_loss")
plt.legend(loc='best')
plt.title('training')
plt.xlabel('epoch')
plt.savefig(TRAINING_PLOT_FILE, bbox_inches='tight')
plt.show()

plt.plot(epochs, history.history['acc'], color='blue', label="training_accuracy")
plt.plot(epochs, history.history['val_acc'], color='red',label="validation_accuracy")
plt.legend(loc='best')
plt.title('validation')
plt.xlabel('epoch')
plt.savefig(VALIDATION_PLOT_FILE, bbox_inches='tight')
plt.show()